In [3]:

import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("/content/train.csv")

# Remove units and convert to float
def clean_mileage(mileage):
    if pd.isna(mileage): return np.nan
    return float(mileage.split()[0])

def clean_engine(engine):
    if pd.isna(engine): return np.nan
    return int(engine.split()[0])

def clean_power(power):
    if pd.isna(power): return np.nan
    return float(power.split()[0])

def clean_new_price(price):
    if pd.isna(price): return np.nan
    value, unit = price.split()
    value = float(value.replace(",", ""))
    if unit.lower() == 'cr':
        value *= 100  # convert crore to lakh
    return value

df['Mileage'] = df['Mileage'].apply(clean_mileage)
df['Engine'] = df['Engine'].apply(clean_engine)
df['Power'] = df['Power'].apply(clean_power)
df['New_Price'] = df['New_Price'].apply(clean_new_price)
df.to_csv("task_b_units_removed.csv", index=False)
# Now all 4 columns are numeric and ready for missing value handling
print(df[['Mileage', 'Engine', 'Power', 'New_Price']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Mileage    5845 non-null   float64
 1   Engine     5811 non-null   float64
 2   Power      5811 non-null   float64
 3   New_Price  815 non-null    float64
dtypes: float64(4)
memory usage: 182.8 KB
None


In [4]:
# Check missing values
missing = df.isnull().sum().sort_values(ascending=False)
print("Missing values:\n", missing)


Missing values:
 New_Price            5032
Seats                  38
Power                  36
Engine                 36
Mileage                 2
Unnamed: 0              0
Fuel_Type               0
Kilometers_Driven       0
Year                    0
Location                0
Name                    0
Owner_Type              0
Transmission            0
Price                   0
dtype: int64


In [5]:
# Drop 'New_Price' due to excessive missing values
df.drop(columns=['New_Price'], inplace=True)

# Impute remaining missing values
df['Seats'].fillna(df['Seats'].mode()[0], inplace=True)
df['Power'].fillna(df['Power'].median(), inplace=True)
df['Engine'].fillna(df['Engine'].median(), inplace=True)
df['Mileage'].fillna(df['Mileage'].median(), inplace=True)
df.to_csv("task_a_missing_values.csv", index=False)

<ipython-input-5-935c42de9247>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Seats'].fillna(df['Seats'].mode()[0], inplace=True)
<ipython-input-5-935c42de9247>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

**Justification**:

The column 'New_Price' was removed as it contained a high amount of missing values—5032 of the total rows. If a column contains over 40–50% missing data, then it typically doesn't contain much value for analysis or modeling. Retaining such a column can contribute additional noise, so it was removed to maintain model quality and reliability.

For the 'Seats' column, imputation was performed using the mode since it is a categorical variable. The mode is the most frequently occurring value in the data and is a reasonable and realistic value that aligns with common vehicle configurations.

Imputation of 'Power', 'Engine', and 'Mileage' was done using the median because they are numerical features that are skewed or contain outliers. The median is less affected by very high or very low values compared to the mean, and thus it is a more robust method that better describes the central tendency of the data in such cases.



In [6]:

# Check for any remaining missing values after imputation
missing_values_after = df.isnull().sum()

# Print the result
print("Missing values after handling:")
print(missing_values_after)

Missing values after handling:
Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"Categorical columns: {categorical_cols}")

# For each categorical column, check its unique values
for col in categorical_cols:
    print(f"Unique values in '{col}': {df[col].unique()}")

Categorical columns: Index(['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type'], dtype='object')
Unique values in 'Name': ['Hyundai Creta 1.6 CRDi SX Option' 'Honda Jazz V' 'Maruti Ertiga VDI' ...
 'Volkswagen Polo IPL II 1.2 Petrol Highline' 'Tata Bolt Revotron XT'
 'Mahindra Xylo D4 BSIV']
Unique values in 'Location': ['Pune' 'Chennai' 'Coimbatore' 'Jaipur' 'Mumbai' 'Kochi' 'Kolkata' 'Delhi'
 'Bangalore' 'Hyderabad' 'Ahmedabad']
Unique values in 'Fuel_Type': ['Diesel' 'Petrol' 'Electric']
Unique values in 'Transmission': ['Manual' 'Automatic']
Unique values in 'Owner_Type': ['First' 'Second' 'Fourth & Above' 'Third']


In [10]:

# One-hot encode the categorical columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True,dtype=int)
df_encoded.to_csv("task_c_hot_encoded.csv", index=False)

# View the new dataframe
print(df_encoded.head())

   Unnamed: 0  Year  Kilometers_Driven  Mileage  Engine   Power  Seats  Price  \
0           1  2015              41000    19.67  1582.0  126.20    5.0  12.50   
1           2  2011              46000    13.00  1199.0   88.70    5.0   4.50   
2           3  2012              87000    20.77  1248.0   88.76    7.0   6.00   
3           4  2013              40670    15.20  1968.0  140.80    5.0  17.74   
4           6  2013              86999    23.08  1461.0   63.10    5.0   3.50   

   Name_Audi A3 35 TDI Attraction  Name_Audi A3 35 TDI Premium  ...  \
0                               0                            0  ...   
1                               0                            0  ...   
2                               0                            0  ...   
3                               0                            0  ...   
4                               0                            0  ...   

   Location_Kochi  Location_Kolkata  Location_Mumbai  Location_Pune  \
0              

In [11]:
from datetime import datetime

# Get the current year
current_year = datetime.now().year

# Create a new 'Age' column
df['Age'] = current_year - df['Year']
df.to_csv("task_d_car_age_column.csv", index=False)
# Check if the 'Age' column was created successfully
print("Columns after adding 'Age' column:")
print(df.columns)

# Display the first few rows to confirm the new 'Age' column
print("\nFirst few rows with the 'Age' column:")
print(df.head())

Columns after adding 'Age' column:
Index(['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven',
       'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power',
       'Seats', 'Price', 'Age'],
      dtype='object')

First few rows with the 'Age' column:
   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Fuel_Type Transmission Owner_Type  Mileage  Engine  \
0              41000    Diesel       Manual      First    19.67  1582.0   
1              46000    Petrol       Manual      First    13.00  1199.0   
2              87000    Diesel       Manual      First    20.77  1248.0   
3     

In [12]:
# Select specific columns
selected_columns = df[['Name', 'Year', 'Fuel_Type', 'Price']]
print(selected_columns.head())
selected_columns.to_csv("task_e_selected_columns.csv", index=False)

                               Name  Year Fuel_Type  Price
0  Hyundai Creta 1.6 CRDi SX Option  2015    Diesel  12.50
1                      Honda Jazz V  2011    Petrol   4.50
2                 Maruti Ertiga VDI  2012    Diesel   6.00
3   Audi A4 New 2.0 TDI Multitronic  2013    Diesel  17.74
4            Nissan Micra Diesel XV  2013    Diesel   3.50


In [13]:

# Rename 'Price' to 'Car_Price' and 'Year' to 'Manufacture_Year'
df.rename(columns={'Price': 'Car_Price', 'Year': 'Manufacture_Year'}, inplace=True)
print(df.columns)

Index(['Unnamed: 0', 'Name', 'Location', 'Manufacture_Year',
       'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'Seats', 'Car_Price', 'Age'],
      dtype='object')


In [14]:
# Mutate (Create a new column) – Calculate price per kilometer driven
df['Price_per_Km'] = df['Car_Price'] / df['Kilometers_Driven']
print(df[['Name', 'Car_Price', 'Kilometers_Driven', 'Price_per_Km']].head())
df[['Name', 'Car_Price', 'Kilometers_Driven', 'Price_per_Km']].to_csv("task_e_mutate_price_per_km.csv", index=False)


                               Name  Car_Price  Kilometers_Driven  \
0  Hyundai Creta 1.6 CRDi SX Option      12.50              41000   
1                      Honda Jazz V       4.50              46000   
2                 Maruti Ertiga VDI       6.00              87000   
3   Audi A4 New 2.0 TDI Multitronic      17.74              40670   
4            Nissan Micra Diesel XV       3.50              86999   

   Price_per_Km  
0      0.000305  
1      0.000098  
2      0.000069  
3      0.000436  
4      0.000040  


In [15]:
# Arrange (sort) by 'Car_Price' in descending order
df_sorted = df.sort_values(by='Car_Price', ascending=False)
print(df_sorted[['Name', 'Car_Price']].head())
df_sorted[['Name', 'Car_Price']].to_csv("task_e_sorted_by_price.csv", index=False)

                                             Name  Car_Price
3952  Land Rover Range Rover 3.0 Diesel LWB Vogue     160.00
5620                   Lamborghini Gallardo Coupe     120.00
5752                       Jaguar F Type 5.0 V8 S     100.00
1457              Land Rover Range Rover Sport SE      97.07
1917                           BMW 7 Series 740Li      93.67


In [16]:
# Group by 'Fuel_Type' and calculate the average price of cars for each fuel type
fuel_group_summary = df.groupby('Fuel_Type')['Car_Price'].mean().reset_index()
print(fuel_group_summary)
fuel_group_summary.to_csv("task_e_groupby_fuel_summary.csv", index=False)

  Fuel_Type  Car_Price
0    Diesel  12.960686
1  Electric  12.875000
2    Petrol   5.756688


In [17]:
# Group by 'Fuel_Type' and 'Transmission' and calculate the average price for each group
fuel_trans_group_summary = df.groupby(['Fuel_Type', 'Transmission'])['Car_Price'].mean().reset_index()
print(fuel_trans_group_summary)
fuel_trans_group_summary.to_csv("task_e_groupby_fuel_trans_summary.csv", index=False)

  Fuel_Type Transmission  Car_Price
0    Diesel    Automatic  24.618490
1    Diesel       Manual   6.686462
2  Electric    Automatic  12.875000
3    Petrol    Automatic  11.252450
4    Petrol       Manual   4.160803
